In [ ]:
!pip install smote-variants
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

from pandas.core.internals.array_manager import new_block
from imblearn.datasets import fetch_datasets

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import smote_variants as sv
###########################
import sys
sys.path.append('/content/cWGAN')
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from dataloader import load_data
from helpers import get_cat_dims

from models import WGANGP

import logging
###########################
sys.path.append('/content/CTAB-GAN')
from ctabgan import CTABGAN
###########################
sys.path.append('/content/SMOTified GAN')
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
from torchvision.datasets.utils import download_url
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader
from GANs_model import GANs_Discriminator, GANs_Generator
from model_trainer import train_discriminator, train_generator
from model_fit import SG_fit, G_fit
from test_model import test_model, test_model_lists
from Abalone_data_preprocessing import two_classes_Abalone, four_classes_Abalone, get_features, get_labels, GANs_two_class_real_data, GANs_four_class_real_data
from choose_device import get_default_device, to_device, DeviceDataLoader
from fit import f1
###########################
sys.path.append('/content/ConvGeN')
from library.dataset import DataSet
from library.generators.ConvGeN import ConvGeN
###########################
sys.path.append('/content/CIGAN')
from CIGAN import CIGAN
###########################
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import RocCurveDisplay

#import os
import random
import tensorflow

#os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"
#os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
#!CUBLAS_WORKSPACE_CONFIG=:4096:8
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True)
tensorflow.random.set_seed(0)

ecoli = fetch_datasets()['ecoli']
abalone_19 = fetch_datasets()['abalone_19']
abalone = fetch_datasets()['abalone']
spectrometer = fetch_datasets()['spectrometer']
us_crime = fetch_datasets()['us_crime']

dataset = us_crime

rate = 1.5

X = np.array(dataset["data"])
y = np.array(dataset["target"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#df = pd.DataFrame(dataset.data)
#dft = pd.DataFrame(dataset.target)
#dfc = pd.concat([df, dft], axis=1)
#dfc.columns = [*dfc.columns[:-1], df.shape[1]]
#dfc.to_csv('/content/dataset.csv', index=False)

u, c = np.unique(y_train, return_counts = True)
minority_class = u[0]
majority_class = u[1]
minority_class_count = c[0]
majority_class_count = c[1]
if c[1] < c[0]:
  minority_class = u[1]
  majority_class = u[0]
  minority_class_count = c[1]
  majority_class_count = c[0]
  
def test_SMOTE(X_train, y_train):
  X_resampled, y_resampled = sv.SMOTE(random_state=0, nn_params={'metric': 'euclidean'}, proportion=rate).sample(X_train, y_train)
  return X_resampled, y_resampled
def test_Borderline_SMOTE1(X_train, y_train):
  X_resampled, y_resampled = sv.Borderline_SMOTE1(random_state=0, nn_params={'metric': 'euclidean'}, proportion=rate).sample(X_train, y_train)
  return X_resampled, y_resampled
def test_Borderline_SMOTE2(X_train, y_train):
  X_resampled, y_resampled = sv.Borderline_SMOTE2(random_state=0, nn_params={'metric': 'euclidean'}, proportion=rate).sample(X_train, y_train)
  return X_resampled, y_resampled
def test_ADASYN(X_train, y_train):
  X_resampled, y_resampled = sv.ADASYN(random_state=0, nn_params={'metric': 'euclidean'}, proportion=rate).sample(X_train, y_train)
  return X_resampled, y_resampled
def test_AND_SMOTE(X_train, y_train):
  X_resampled, y_resampled = sv.AND_SMOTE(random_state=0, nn_params={'metric': 'euclidean'}, proportion=rate).sample(X_train, y_train)
  return X_resampled, y_resampled
def test_SOI_CJ(X_train, y_train):
  X_resampled, y_resampled = sv.SOI_CJ(random_state=0, nn_params={'metric': 'euclidean'}, proportion=rate).sample(X_train, y_train)
  return X_resampled, y_resampled

X_resampled_a, y_resampled_a = test_SMOTE(X_train, y_train)
X_resampled_b1, y_resampled_b1 = test_Borderline_SMOTE1(X_train, y_train)
X_resampled_b2, y_resampled_b2 = test_Borderline_SMOTE2(X_train, y_train)
X_resampled_c, y_resampled_c = test_ADASYN(X_train, y_train)
X_resampled_d, y_resampled_d = test_AND_SMOTE(X_train, y_train)
X_resampled_e, y_resampled_e = test_SOI_CJ(X_train, y_train)

#X_resampled_a, y_resampled_a = sv.SMOTE(random_state=0).sample(X_train, y_train)
#X_resampled_b1, y_resampled_b1 = sv.Borderline_SMOTE1(random_state=0).sample(X_train, y_train)
#X_resampled_b2, y_resampled_b2 = sv.Borderline_SMOTE2(random_state=0).sample(X_train, y_train)
#X_resampled_c, y_resampled_c = sv.ADASYN(random_state=0).sample(X_train, y_train)
#X_resampled_d, y_resampled_d = sv.AND_SMOTE(random_state=0).sample(X_train, y_train)
#X_resampled_e, y_resampled_e = sv.SOI_CJ(random_state=0).sample(X_train, y_train)
##################################################cWGAN
def prep_for_cWGAN():
  # preprocess data
  num_prep = make_pipeline(SimpleImputer(strategy='mean'),
                          MinMaxScaler())
  cat_prep = make_pipeline(SimpleImputer(strategy='most_frequent'),
                          OneHotEncoder(handle_unknown='ignore', sparse=False))
  num_cols = [i for i in range(X_train.shape[1])]
  cat_cols = []
  prep = ColumnTransformer([
      ('num', num_prep, num_cols),
      ('cat', cat_prep, cat_cols)],
      remainder='drop')
  return prep
def test_cWGANGP(X_train, y_train):
  #to make no_oversampling working properly
  u, c = np.unique(y_train, return_counts = True)
  minority_class = u[0]
  majority_class = u[1]
  minority_class_count = c[0]
  majority_class_count = c[1]
  if c[1] < c[0]:
    minority_class = u[1]
    majority_class = u[0]
    minority_class_count = c[1]
    majority_class_count = c[0]

  num_cols = [i for i in range(X_train.shape[1])]
  cat_cols = []
  prep = prep_for_cWGAN()
  X_train_trans = prep.fit_transform(X_train)
  y_train_trans = np.where(y_train == 1, 1, 0)
  gan = WGANGP(write_to_disk=True, # whether to create an output folder. Plotting will be surpressed if flase
              compute_metrics_every=1250, print_every=2500, plot_every=1000000,
              num_cols = num_cols, cat_dims=[],
              # pass the one hot encoder to the GAN to enable count plots of categorical variables
              transformer=prep.named_transformers_['cat']['onehotencoder'],
              # pass column names to enable
              cat_cols=cat_cols,
              use_aux_classifier_loss=True,
              d_updates_per_g=3, gp_weight=15)
  gan.fit(X_train_trans, y=y_train_trans, 
          condition=True,
          epochs=300,  
          batch_size=64,
          netG_kwargs = {'hidden_layer_sizes': (128,64), 
                          'n_cross_layers': 1,
                          'cat_activation': 'gumbel_softmax',
                          'num_activation': 'none',
                          'condition_num_on_cat': False,#True, 
                          'noise_dim': 30, 
                          'normal_noise': False,
                          'activation':  'leaky_relu',
                          'reduce_cat_dim': True,
                          'use_num_hidden_layer': True,
                          'layer_norm':False,},
          netD_kwargs = {'hidden_layer_sizes': (128,64,32),
                          'n_cross_layers': 2,
                          'embedding_dims': 'auto',
                          'activation':  'leaky_relu',
                          'sigmoid_activation': False,
                          'noisy_num_cols': True,
                          'layer_norm':True,}
        )
  X_resampled_1, y_resampled_1 = gan.resample(X_train_trans, y_train_trans, int((majority_class_count - minority_class_count)*rate))
  y_resampled_1 = np.where(y_resampled_1 == 1, 1, -1)
  return X_resampled_1, y_resampled_1
X_resampled_1, y_resampled_1 = test_cWGANGP(X_train, y_train)
##################################################
##################################################CTAB-GAN
no_oversampling_test_split = False
def test_CTABGAN(X_train, y_train):
  df = pd.DataFrame(X_train)
  dft = pd.DataFrame(y_train)
  dfc = pd.concat([df, dft], axis=1)
  dfc.columns = [*dfc.columns[:-1], df.shape[1]]
  dfc.to_csv('/content/dataset.csv', index=False)

  num_cols = [i for i in range(X_train.shape[1])]
  synthesizer =  CTABGAN(raw_csv_path = '/content/dataset.csv',
                  test_ratio = 0,  
                  categorical_columns = [], 
                  log_columns = [],
                  mixed_columns= {}, 
                  integer_columns = num_cols,
                  problem_type= {"Classification": str(df.shape[1])},
                  epochs = 150) 

  #to make no_oversampling working properly
  u, c = np.unique(y_train, return_counts = True)
  minority_class = u[0]
  majority_class = u[1]
  minority_class_count = c[0]
  majority_class_count = c[1]
  if c[1] < c[0]:
    minority_class = u[1]
    majority_class = u[0]
    minority_class_count = c[1]
    majority_class_count = c[0]
  # Fitting the synthesizer to the training dataset and generating synthetic data
  X_resampled_2 = X_train
  new_minority_class_count = minority_class_count
  ultimate_minority_class_count = int((majority_class_count - minority_class_count)*rate + minority_class_count)
  synthesizer.fit()
  while (new_minority_class_count<ultimate_minority_class_count):
      syn = synthesizer.generate_samples()
      syn[str(df.shape[1])] = syn[str(df.shape[1])].round()
      syn[str(df.shape[1])] = syn[str(df.shape[1])].apply(lambda x: int(x) if(x == x) else None)
      new_minority = syn[syn[str(df.shape[1])]==minority_class]
      new_minority = new_minority.drop(columns=[str(df.shape[1])])
      new_minority_class_count += new_minority.shape[0]
      if (new_minority_class_count > ultimate_minority_class_count):
        diff = new_minority_class_count-ultimate_minority_class_count
        new_minority = new_minority.drop(new_minority.tail(diff).index)
        X_resampled_2 = np.vstack((X_resampled_2, new_minority.to_numpy()))
        break
      X_resampled_2 = np.vstack((X_resampled_2, new_minority.to_numpy()))
  y_resampled_2 = np.concatenate([y_train, np.full((X_resampled_2.shape[0]-X_train.shape[0]), minority_class)])
  return X_resampled_2, y_resampled_2
X_resampled_2, y_resampled_2 = test_CTABGAN(X_train, y_train)
##################################################
##################################################SMOTified-GAN
def test_SMOTified_GAN(X_train, y_train):
  def shuffle_in_unison(a, b):     #Shuffling the features and labels in unison.
      assert len(a) == len(b)       #In Python, the assert statement is used to continue the execute if the given condition evaluates to True.
      shuffled_a = np.empty(a.shape, dtype=a.dtype)       #Return a new array of given shape and type, without initializing entries.
      shuffled_b = np.empty(b.shape, dtype=b.dtype)
      permutation = np.random.permutation(len(a))
      for old_index, new_index in enumerate(permutation):
          shuffled_a[new_index] = a[old_index]
          shuffled_b[new_index] = b[old_index]
      return shuffled_a, shuffled_b

  def GANs_real_data(X_train, y_train, minority_class):   #Defining the real data for GANs
    X_real = []
    y_train = y_train.ravel()
    for i in range(len(y_train)):
      if int(y_train[i])==minority_class:
        X_real.append(X_train[i])
    X_real = np.array(X_real)
    y_real = np.full((X_real.shape[0],), minority_class)
    return X_real, y_real

  #to make no_oversampling working properly
  u, c = np.unique(y_train, return_counts = True)
  minority_class = u[0]
  majority_class = u[1]
  minority_class_count = c[0]
  majority_class_count = c[1]
  if c[1] < c[0]:
    minority_class = u[1]
    majority_class = u[0]
    minority_class_count = c[1]
    majority_class_count = c[0]

  #special conditions for rates: 0.5, 1.0, 1.5
  if rate==0.5:
    X_train_SMOTE,y_train_SMOTE = SMOTE(sampling_strategy=rate+(minority_class_count/majority_class_count)*rate, random_state=0).fit_resample(X_train,y_train)
  elif rate==1.0:
    X_train_SMOTE,y_train_SMOTE = SMOTE(sampling_strategy=rate, random_state=0).fit_resample(X_train,y_train)
  elif rate==1.5:
    X_train_SMOTE_a,y_train_SMOTE_a = SMOTE(sampling_strategy=1.0, random_state=0).fit_resample(X_train,y_train)
    X_train_SMOTE_b,y_train_SMOTE_b = SMOTE(sampling_strategy=0.5-(minority_class_count/majority_class_count)*0.5, random_state=0).fit_resample(X_train,y_train)
    X_train_SMOTE_b = X_train_SMOTE_b[y_train_SMOTE_b[:]==minority_class]
    y_train_SMOTE_b = y_train_SMOTE_b[y_train_SMOTE_b[:]==minority_class]
    X_train_SMOTE = np.concatenate((X_train_SMOTE_a, X_train_SMOTE_b), axis=0)
    y_train_SMOTE = np.concatenate((y_train_SMOTE_a, y_train_SMOTE_b), axis=0)
  else:
    X_train_SMOTE,y_train_SMOTE = SMOTE(sampling_strategy=rate, random_state=0).fit_resample(X_train,y_train)
  device = get_default_device()

  ##### Oversampled data from SMOTE that is now to be passed in SMOTified GANs #####
  X_oversampled = X_train_SMOTE[(X_train.shape[0]):]
  X_oversampled = torch.from_numpy(X_oversampled)
  X_oversampled = to_device(X_oversampled.float(), device)

  lr = 0.0002
  epochs = 150
  batch_size = 128

  X_real, y_real = GANs_real_data(X_train, y_train, minority_class)   #Defining the real data to be put in GANs

  #Training our SMOTified GANs and GANs model and fetching their trained generators.
  generator_SG, generator_G = f1(X_train, y_train, X_train_SMOTE, y_train_SMOTE, X_real, y_real, X_oversampled, device, lr, epochs, batch_size, minority_class, majority_class)

  Trained_X_oversampled_SG = generator_SG(X_oversampled.float().to(device)).cpu().detach().numpy()
  Trained_SG_dataset = np.concatenate((X_train_SMOTE[:(X_train.shape[0])], Trained_X_oversampled_SG), axis=0)
  X_resampled_3, y_resampled_3 = shuffle_in_unison(Trained_SG_dataset, y_train_SMOTE)
  return X_resampled_3, y_resampled_3
X_resampled_3, y_resampled_3 = test_SMOTified_GAN(X_train, y_train)
##################################################
##################################################ConvGeN
def test_ConvGeN(X_train, y_train):
  def loadData(X_train, y_train):
    features = X_train
    labels = y_train
    label_1 = list(np.where(labels == 1)[0])
    label_0 = list(np.where(labels != 1)[0])
    features_1 = features[label_1]
    features_0 = features[label_0]
    return DataSet(data0=features_0, data1=features_1)

  #to make no_oversampling working properly
  u, c = np.unique(y_train, return_counts = True)
  minority_class = u[0]
  majority_class = u[1]
  minority_class_count = c[0]
  majority_class_count = c[1]
  if c[1] < c[0]:
    minority_class = u[1]
    majority_class = u[0]
    minority_class_count = c[1]
    majority_class_count = c[0]

  data = loadData(X_train, y_train)
  gen = ConvGeN(data.data0.shape[1], neb=5)#, neb_epochs=3)
  gen.reset(data)
  gen.train(data)
  syntheticPoints = gen.generateData(int((majority_class_count - minority_class_count)*rate))

  X_resampled_4 = np.vstack((X_train, syntheticPoints))
  y_resampled_4 = np.concatenate([y_train, np.full((X_resampled_4.shape[0]-X_train.shape[0]), minority_class)])
  return X_resampled_4, y_resampled_4
X_resampled_4, y_resampled_4 = test_ConvGeN(X_train, y_train)
##################################################
##################################################CIGAN
def test_CIGAN(X_train, y_train):
  #to make no_oversampling working properly
  u, c = np.unique(y_train, return_counts = True)
  minority_class = u[0]
  majority_class = u[1]
  minority_class_count = c[0]
  majority_class_count = c[1]
  if c[1] < c[0]:
    minority_class = u[1]
    majority_class = u[0]
    minority_class_count = c[1]
    majority_class_count = c[0]

  # The CIGAN
  cigan = CIGAN(X_train, y_train, int((majority_class_count - minority_class_count)*rate), generator_learning_rate=10 ** -4, discriminator_learning_rate=10 ** -4)

  # Augment the minority classes in the training data
  X_resampled_5, y_resampled_5 = cigan.fit_resample(X_train, y_train)
  return X_resampled_5, y_resampled_5
X_resampled_5, y_resampled_5 = test_CIGAN(X_train, y_train)
##################################################
def check_no_oversampling(X_train, y_train, minority_class, majority_class):
  def HasD(x, y):
    total = 0
    for xi, yi in zip(x, y):
        min_value = min(xi, yi)
        max_value = max(xi, yi)
        total += 1 # we sum the 1 in both cases
        if min_value >= 0:
            total -= (1 + min_value) / (1 + max_value)
        else:
            # min_value + abs(min_value) = 0, so we ignore that
            total -= 1 / (1 + max_value + abs(min_value))
    return total

  minoridx=np.where(y_train==minority_class)[0]
  majoridx=np.where(y_train==majority_class)[0]
  Xminor=X_train[minoridx,:]
  Xmajor=X_train[majoridx,:]
  yminor=y_train[minoridx]
  ymajor=y_train[majoridx]

  print(f'The data has {Xminor.shape[0]}  minority samples, labeled {(minority_class)}, and {Xmajor.shape[0]} majority samples, labeled {(majority_class)}')
  print(Xminor.shape[1])

  minoritypercent=0.0
  majoritypercent=0.50

  Nminortoremove=np.fix(len(Xminor)*minoritypercent).astype(np.int64)
  Nmajortoremove=np.fix(len(Xmajor)*majoritypercent).astype(np.int64)

  print(f'{Nminortoremove} from minority and {Nmajortoremove} from majority will be removed')

  Xhiddenmajor=Xmajor[0:Nmajortoremove,:]
  yhiddenmajor=ymajor[0:Nmajortoremove]

  Xnewminor=Xminor[Nminortoremove:]
  Xnewmajor=Xmajor[Nmajortoremove:]
  ynewminor=yminor[Nminortoremove:]
  ynewmajor=ymajor[Nmajortoremove:]  

  print(f'the number of hidden examples is {Xhiddenmajor.shape[0]}')

  XnewData=np.vstack((Xnewmajor,Xnewminor))
  ynewData=np.hstack((ynewmajor,ynewminor))
  sizeofnewdata=XnewData.shape[0]

  # stack the hidden data with the original to find the similarity 
  XKNN=np.vstack((XnewData,Xhiddenmajor))
  yKNN=np.hstack((ynewData,yhiddenmajor))

  print(f'The new data after removing samples has {sizeofnewdata} samples, {Xnewminor.shape[0]} minority and {Xnewmajor.shape[0]} majority ')

  SuccNames=[]
  ErrorPercentages=[]
  ErrorCounts=[]
  methods = [test_SMOTE, test_Borderline_SMOTE1, test_Borderline_SMOTE2, test_ADASYN,
             test_AND_SMOTE, test_SOI_CJ,test_cWGANGP, test_CTABGAN,
             test_SMOTified_GAN, test_ConvGeN, test_CIGAN]
  string_a = []
  string_b = []
  for methodidx in methods:
      try:        
        X_samp, y_samp= methodidx(XnewData, ynewData)
        resambledX=X_samp[sizeofnewdata:]
        resambledy=y_samp[sizeofnewdata:]
        #string_a.append(f'The number of added samples is {resambledX.shape[0]}')
        #print(f'The number of added samples is {resambledX.shape[0]}')
        neigh = KNeighborsClassifier(n_neighbors=1, metric=HasD)
        neigh.fit(XKNN, yKNN)

        predis=neigh.predict(resambledX)
        predis==majority_class
        countwrong=np.count_nonzero(predis==majority_class)
        Errorpercent= countwrong/len(predis)
        string_a.append(f'The number of added samples is {resambledX.shape[0]}, X_samp is {X_samp.shape[0]} and sizeofnewdata is {sizeofnewdata}')
        string_b.append(f'The error percent of {methodidx.__name__} is {Errorpercent}, with number of wrong predictions = {countwrong} out of {resambledX.shape[0]}')
        #print(f'The error percent of {methodidx.__name__} is {Errorpercent}, with number of wrong predictions = {countwrong} out of {resambledX.shape[0]}')
        ErrorPercentages.append(Errorpercent)
        ErrorCounts.append(countwrong)
        SuccNames.append(methodidx.__name__)

      except:
        print("EXCEPT")
        #print(f'cannot perform using {MethodName}')
        #writer.writerow([MethodName,'-','-','-','-','-','-','-','-'])
  for a, b in zip(string_a, string_b):
    print(a)
    print(b)
##################################################
no_oversampling_test_split = True
check_no_oversampling(X_train, y_train, minority_class, majority_class)

plt.rcParams.update({'font.size': 12})
fig, ax =plt.subplots(4,3, figsize=(25,25))
tsne = TSNE(n_components=2, random_state=0)

def shape(X, y):
  features = X
  labels = y
  label_1 = list(np.where(labels == 1)[0])
  label_0 = list(np.where(labels != 1)[0])
  features_1 = features[label_1]
  features_0 = features[label_0]
  print(features_1.shape)
  print(features_0.shape)

def results(X_resampled, y_resampled, X_test, y_test, cls, name):
  cls.fit(X_resampled, y_resampled)
  y_pred = cls.predict(X_test)
  print(name)
  print(classification_report_imbalanced(y_test, y_pred))
  #print(precision_recall_fscore_support(y_test, y_pred))
  print('f1_score')
  print(f1_score(y_test, y_pred))
  print('geometric_mean_score')
  print(geometric_mean_score(y_test, y_pred))
  print('accuracy_score')
  print(accuracy_score(y_test, y_pred))
  print('balanced_accuracy_score')
  print(balanced_accuracy_score(y_test, y_pred))
  RocCurveDisplay.from_estimator(cls, X_test, y_test, name = cls.__class__)
  #RocCurveDisplay.from_predictions(y_test, y_pred, name = cls.__class__)

sns_i = 0
def first_results(X_resampled, y_resampled, X_test, y_test, cls, name):
  global sns_i
  z = tsne.fit_transform(X_resampled)
  sns.scatterplot(data=z, x=z[:,0], y=z[:,1], hue=y_resampled, palette=sns.color_palette("hls", 3), ax=ax[int(sns_i/3)][sns_i%3]).set(title=name, xlabel="Feature 1", ylabel="Feature 2")
  shape(X_resampled, y_resampled)
  sns_i = sns_i + 1
  results(X_resampled, y_resampled, X_test, y_test, cls, name)

cWGAN_prep = prep_for_cWGAN()
cWGAN_prep.fit_transform(X_train)
cls = KNeighborsClassifier()
first_results(X_train, y_train, X_test, y_test, cls, 'No oversampling')
first_results(X_resampled_a, y_resampled_a, X_test, y_test, cls, 'SMOTE')
first_results(X_resampled_b1, y_resampled_b1, X_test, y_test, cls, 'Borderline-SMOTE1')
first_results(X_resampled_b2, y_resampled_b2, X_test, y_test, cls, 'Borderline-SMOTE2')
first_results(X_resampled_c, y_resampled_c, X_test, y_test, cls, 'ADASYN')
first_results(X_resampled_d, y_resampled_d, X_test, y_test, cls, 'AND SMOTE')
first_results(X_resampled_e, y_resampled_e, X_test, y_test, cls, 'SOI-CJ')
first_results(X_resampled_1, y_resampled_1, cWGAN_prep.transform(X_test), y_test, cls, 'cWGAN')
first_results(X_resampled_2, y_resampled_2, X_test, y_test, cls, 'CTAB-GAN')
first_results(X_resampled_3, y_resampled_3, X_test, y_test, cls, 'SMOTified GAN')
first_results(X_resampled_4, y_resampled_4, X_test, y_test, cls, 'ConvGeN')
first_results(X_resampled_5, y_resampled_5, X_test, y_test, cls, 'CIGAN')

cls = SVC(random_state=0)
results(X_train, y_train, X_test, y_test, cls, 'No oversampling')
results(X_resampled_a, y_resampled_a, X_test, y_test, cls, 'SMOTE')
results(X_resampled_b1, y_resampled_b1, X_test, y_test, cls, 'Borderline-SMOTE1')
results(X_resampled_b2, y_resampled_b2, X_test, y_test, cls, 'Borderline-SMOTE2')
results(X_resampled_c, y_resampled_c, X_test, y_test, cls, 'ADASYN')
results(X_resampled_d, y_resampled_d, X_test, y_test, cls, 'AND SMOTE')
results(X_resampled_e, y_resampled_e, X_test, y_test, cls, 'SOI-CJ')
results(X_resampled_1, y_resampled_1, cWGAN_prep.transform(X_test), y_test, cls, 'cWGAN')
results(X_resampled_2, y_resampled_2, X_test, y_test, cls, 'CTAB-GAN')
results(X_resampled_3, y_resampled_3, X_test, y_test, cls, 'SMOTified GAN')
results(X_resampled_4, y_resampled_4, X_test, y_test, cls, 'ConvGeN')
results(X_resampled_5, y_resampled_5, X_test, y_test, cls, 'CIGAN')

cls = GaussianNB()
results(X_train, y_train, X_test, y_test, cls, 'No oversampling')
results(X_resampled_a, y_resampled_a, X_test, y_test, cls, 'SMOTE')
results(X_resampled_b1, y_resampled_b1, X_test, y_test, cls, 'Borderline-SMOTE1')
results(X_resampled_b2, y_resampled_b2, X_test, y_test, cls, 'Borderline-SMOTE2')
results(X_resampled_c, y_resampled_c, X_test, y_test, cls, 'ADASYN')
results(X_resampled_d, y_resampled_d, X_test, y_test, cls, 'AND SMOTE')
results(X_resampled_e, y_resampled_e, X_test, y_test, cls, 'SOI-CJ')
results(X_resampled_1, y_resampled_1, cWGAN_prep.transform(X_test), y_test, cls, 'cWGAN')
results(X_resampled_2, y_resampled_2, X_test, y_test, cls, 'CTAB-GAN')
results(X_resampled_3, y_resampled_3, X_test, y_test, cls, 'SMOTified GAN')
results(X_resampled_4, y_resampled_4, X_test, y_test, cls, 'ConvGeN')
results(X_resampled_5, y_resampled_5, X_test, y_test, cls, 'CIGAN')

cls = DecisionTreeClassifier(random_state=0)
results(X_train, y_train, X_test, y_test, cls, 'No oversampling')
results(X_resampled_a, y_resampled_a, X_test, y_test, cls, 'SMOTE')
results(X_resampled_b1, y_resampled_b1, X_test, y_test, cls, 'Borderline-SMOTE1')
results(X_resampled_b2, y_resampled_b2, X_test, y_test, cls, 'Borderline-SMOTE2')
results(X_resampled_c, y_resampled_c, X_test, y_test, cls, 'ADASYN')
results(X_resampled_d, y_resampled_d, X_test, y_test, cls, 'AND SMOTE')
results(X_resampled_e, y_resampled_e, X_test, y_test, cls, 'SOI-CJ')
results(X_resampled_1, y_resampled_1, cWGAN_prep.transform(X_test), y_test, cls, 'cWGAN')
results(X_resampled_2, y_resampled_2, X_test, y_test, cls, 'CTAB-GAN')
results(X_resampled_3, y_resampled_3, X_test, y_test, cls, 'SMOTified GAN')
results(X_resampled_4, y_resampled_4, X_test, y_test, cls, 'ConvGeN')
results(X_resampled_5, y_resampled_5, X_test, y_test, cls, 'CIGAN')

plt.show()

"""
neigh = KNeighborsClassifier()
neigh.fit(X_resampled, y_resampled)
y_pred = neigh.predict(X_test)

svc = SVC(random_state=0)
svc.fit(X_resampled, y_resampled)
y_pred = svc.predict(X_test)

gnb = GaussianNB()
gnb.fit(X_resampled, y_resampled)
y_pred = gnb.predict(X_test)#X_test_trans
"""